In [11]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt





In [12]:
import torch
import torch.nn as nn
from torchdiffeq import odeint

# -----------------------------
# LTC Cell
# -----------------------------
class LTCCell(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.W = nn.Linear(input_dim + hidden_dim, hidden_dim)
        self.time_constant = nn.Parameter(torch.ones(hidden_dim))

    def forward(self, t, h):
        # h: (batch, hidden_dim)
        x = self.current_input  # will be set externally
        combined = torch.cat([x, h], dim=-1)
        dh = torch.tanh(self.W(combined)) - h
        return dh / (torch.abs(self.time_constant) + 1e-6)

# -----------------------------
# Stacked LTC Model
# -----------------------------
class StackedLTC(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, t_span=[0.0, 1.0]):
        super().__init__()
        self.layers = nn.ModuleList()
        self.t_span = t_span
        prev_dim = input_dim
        for hdim in hidden_dims:
            self.layers.append(LTCCell(prev_dim, hdim))
            prev_dim = hdim
        self.head = nn.Linear(prev_dim, output_dim)

    def forward(self, x):
        device = x.device
        batch_size, seq_len, _ = x.size()
        hs = [torch.zeros(batch_size, layer.hidden_dim, device=device) for layer in self.layers]

        for t in range(seq_len):
            cur = x[:, t, :]
            for li, layer in enumerate(self.layers):
                if li == 0:
                    layer.current_input = cur
                else:
                    layer.current_input = hs[li-1]
                # ensure t_span is tensor
                t_span = torch.tensor(self.t_span, dtype=torch.float32, device=cur.device)
                h_next = odeint(layer, hs[li], t_span)[-1]
                hs[li] = torch.tanh(h_next)

        final_h = hs[-1]
        out = self.head(final_h)
        return out


In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = StackedLTC(input_dim=len(features), hidden_dims=[64, 64, 32], output_dim=1).to(DEVICE)
model.load_state_dict(torch.load("lnn_final_model.pth", map_location=DEVICE))
model.eval()

NameError: name 'features' is not defined